In [47]:
import os
import polars as pl
import pandas as pd
pl.Config(tbl_rows=100)
from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu

In [49]:
df = pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet"))
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","leader.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")
df = df.to_pandas()
df = df[df["leader"].str[6].isin(["a", "t"])]
df = df[~df["leader"].str[7].isin(["b", "i", "s", " "])]
df = df[(df["008"].str[15:17] == "xr") & (df["008"].str[35:38] == "cze")]
df = pl.from_pandas(df)
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","022.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","300.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","655.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","700.parquet")), left_on="001", right_on="001", how="left")
df = df.explode("022_a").filter(pl.col("022_a").is_null())
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))
df = df.with_columns(pl.col('300_a').map_elements(pocet_stran, return_dtype=int).alias('stran'))
df = df.with_columns(pl.col('245_a').map_elements(bezbordelu, return_dtype=str))
df = df.explode('245_p').with_columns(pl.col('245_p').map_elements(bezbordelu, return_dtype=str))
print(len(df))
df = df.filter(pl.col("rok") >= 1800)
df = df.filter((~pl.col("245_h").str.contains("grafika")) | pl.col("245_h").is_null()).unique(subset=["008","100_a","245_a","245_p"], keep="first")
print(len(df))

716107


705250


In [51]:
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","041.parquet")), left_on="001", right_on="001", how="left")

In [52]:
df = df.drop_nulls(subset=["700_a","700_4","700_7"]).explode(["700_a","700_4","700_7"])

In [53]:
df = df.filter(pl.col('700_4') == 'trl')

In [54]:
df.group_by('700_a').agg(pl.col("100_a").n_unique()).sort(by="100_a",descending=True)

700_a,100_a
str,u32
"""Vyskočil, Josef""",187
"""Podaný, Richard,""",145
"""Pokorný, Martin,""",121
"""Štorkánová, Anna,""",115
"""Volhejnová, Veronika,""",105
"""Kufnerová, Zlata,""",102
"""Kadlecová, Eva,""",100
"""Kozák, Jan,""",100
"""Jašová, Jana,""",92


In [55]:
df.group_by('700_a').agg(pl.struct(["245_a","245_p"]).n_unique()).sort(by="245_a",descending=True)

700_a,245_a
str,u32
"""Butala, Tomáš,""",504
"""Pavka, Marek,""",427
"""Vyskočil, Josef""",409
"""Podaný, Richard,""",265
"""Volhejnová, Veronika,""",264
"""Sladká, Stanislava""",246
"""Schönová, Zuzana""",234
"""Pacnerová, Jana,""",225
"""Kuťák, Jaroslav,""",214


In [56]:
df.filter(pl.col('700_a') == "Butala, Tomáš,").group_by("100_a").len()

100_a,len
str,u32
"""Hayes, Rex,""",12
"""Gringel, Dana,""",1
"""Czernin, Monika,""",1
"""Arnaldur Indriðason,""",4
"""Callahan, Frank""",2
"""Taylor, Kathryn""",3
"""Roberts, Dan,""",1
"""Glaser, Frank""",9
"""Morton, Jack""",6


In [57]:
df.filter(pl.col('700_a') == "Butala, Tomáš,").group_by("041_h").len()

041_h,len
list[str],u32
null,1
"[""eng""]",3
"[""ger""]",624
"[""ice""]",4
"[""fre""]",1


In [58]:
df.filter(pl.col('700_a') == "Volhejnová, Veronika,").group_by("rok").len().sort(by="rok")

rok,len
i64,u32
1993,2
1994,2
1995,4
1996,6
1997,11
1998,6
1999,7
2000,3
2001,7


In [59]:
df.group_by('700_a').agg(pl.col("stran").sum()).sort(by="stran",descending=True)

700_a,stran
str,i64
"""Volhejnová, Veronika,""",104086
"""Pacnerová, Jana,""",90284
"""Kantůrek, Jan,""",80551
"""Podaný, Richard,""",75994
"""Dušek, Zdík,""",68357
"""Klůfová, Petra,""",68070
"""Jašová, Jana,""",66951
"""Chodilová, Dana,""",56060
"""Medek, Pavel,""",54195


In [60]:
df.explode("041_h").filter(pl.col('700_4') == 'trl').group_by('700_a').agg(pl.col("041_h").n_unique()).sort(by="041_h",descending=True)

700_a,041_h
str,u32
"""Babler, Otto František,""",19
"""Hiršal, Josef,""",16
"""Vetti, O. S.,""",16
"""Bednář, Kamil,""",12
"""Vrchlický, Jaroslav,""",12
"""Fischer, Otokar,""",11
"""Žáček, Jiří,""",11
"""Kosterka, Hugo,""",11
"""Sýs, Karel,""",11


In [61]:
df.filter(pl.col('700_a') == "Babler, Otto František,").group_by('041_h').agg(pl.col("245_a").n_unique()).sort(by="245_a",descending=True)

041_h,245_a
list[str],u32
"[""eng""]",10
"[""fre""]",8
"[""ger""]",7
"[""scr""]",5
"[""hrv""]",3
"[""bul""]",2
"[""ita""]",2
"[""srp""]",2
null,2


In [62]:
df.filter(pl.col('700_a') == "Hiršal, Josef,").group_by('041_h').agg(pl.col("245_a").n_unique()).sort(by="245_a",descending=True)

041_h,245_a
list[str],u32
"[""ger""]",23
"[""spa""]",3
"[""por""]",3
"[""swe""]",2
"[""scr""]",2
"[""fre""]",2
"[""mul""]",1
"[""hun""]",1
"[""per""]",1
